<h1 class="header"><img class="logo" src="../images/socib_logo.png" width="200px"></h1>

<h3> STYLING NOTEBOOK (OPTIONAL)</h3>

In [1]:
from IPython.core.display import HTML
import urllib2
HTML(urllib2.urlopen('https://raw.githubusercontent.com/socib/API_examples/master/style/custom/custom.css').read())

<h3> HANDY PYTHON PACKAGES </h3>

In [1]:
import requests
import json
import numpy as np
from json2html import *
from IPython.display import HTML, display
%matplotlib inline

<h3> PREREQUISITES </h3>

In [4]:
api_key = '' #write you SOCIB api_key
api_url = 'http://api.socib.es'
headers = {
    'accept': 'application/vnd.socib+json',
    'apikey': api_key,
}

<h3> REQUESTING A DATA-PLATFORM'S FILES </h3>

Let's look for a certain type of data-platforms:

In [5]:
end_point = '/data-platforms/'
platform_type = 'Glider'
url_addons = "?type=%s"%(platform_type)
request = requests.get('%s%s%s' % (api_url, end_point,url_addons), headers=headers)
response = json.loads(request.text)

print('Requested url: '+'%s%s%s' % (api_url, end_point,url_addons))
print(" ")
print('%s platforms of type %s has been used in SOCIB deployments '% (response['count'], platform_type))

Requested url: http://api.socib.es/data-platforms/?type=Glider
 
15 platforms of type Glider has been used in SOCIB deployments 


As the pagination is 8, we will have to loop all pages to get the full list of the data-platforms of type glider:

In [6]:
results = response['results']
page = response['next']
while page != None:
    request = requests.get(page, headers=headers)
    response = json.loads(request.text)
    for glider in response['results']:
        results.append(glider)
    page = response['next']

In [7]:
HTML(json2html.convert(json=results))

wmo_naming_authority,wmo_code,type,name,is_active
JCOMMOBS,68968,Glider,icoast00,False
JCOMMOBS,68452,Glider,ideep00,False
JCOMMOBS,68996,Glider,ideep01,False
JCOMMOBS,68966,Glider,ideep02,False
JCOMMOBS,68457,Glider,sdeep00,False
JCOMMOBS,68967,Glider,sdeep01,False
JCOMMOBS,68965,Glider,sdeep02,False
JCOMMOBS,68969,Glider,sdeep03,False
JCOMMOBS,68997,Glider,sdeep04,False
JCOMMOBS,68998,Glider,sdeep05,False


Next we will see how to get latest netCDFs comming form one of the above platforms.

<h3>EXPLORING THE <span>/DATA-PLATFORM/{NAME}/FILES/LATEST</span> AUXILIARY ENDPOINT</h3>

Let's focuse on the first data-platform from above list and request its latest files:

In [8]:
targeted_data_platform = results[0] #data platform number 0

In [9]:
end_point_plus_aux = '/data-platforms/'+targeted_data_platform['name']+'/files/latest'
request = requests.get('%s%s' % (api_url, end_point_plus_aux), headers=headers)
response = json.loads(request.text)
print('Requested url: '+'%s%s. Be aware of the name requested' % (api_url, end_point_plus_aux))

Requested url: http://api.socib.es/data-platforms/icoast00/files/latest. Be aware of the name requested


In [10]:
files_metadata = {k:v for k, v in response.items() if k != 'entries'} #skipping the deployment data for later
for k, v in response.items():
    if k == 'entries':
        files_metadata[k] = len(v) 
HTML(json2html.convert(json=files_metadata))

type,Glider
variables,
is_active,False
name,icoast00
entries,0


<b>By default, the /DATA-PLATFORM/{NAME}/FILES/LATEST auxiliary endpoint pressume that you are interested only in 'current' latest files</b>. If the above platform is inactive, the number of netCDFs ('entries') is zero, while if it is active, the netCDFs from the last deployment are retourned.

Let's see how to overwrite the above defaulting behaviour to get current or simply last available files:

<h3>/DATA-PLATFORM/{NAME}/FILES/LATEST WITH PARAMS <span>CURRENT FILES</span></h3>

In [11]:
end_point_plus_aux = '/data-platforms/'+targeted_data_platform['name']+'/files/latest'
url_addons = '/?current=true'
request = requests.get('%s%s%s' % (api_url, end_point_plus_aux, url_addons), headers=headers)
response = json.loads(request.text)
print('Requested url: '+'%s%s%s. Be aware of the name requested' % (api_url, end_point_plus_aux, url_addons))

Requested url: http://api.socib.es/data-platforms/icoast00/files/latest/?current=true. Be aware of the name requested


In [12]:
files_metadata = {k:v for k, v in response.items() if k != 'entries'} #skipping the deployment data for later
for k, v in response.items():
    if k == 'entries':
        files_metadata[k] = len(v) 
HTML(json2html.convert(json=files_metadata))

type,Glider
variables,
is_active,False
name,icoast00
entries,0


<h3>/DATA-PLATFORM/{NAME}/FILES/LATEST WITH PARAMS <span>LAST AVAILABLE FILES</span></h3>

In [13]:
end_point_plus_aux = '/data-platforms/'+targeted_data_platform['name']+'/files/latest'
url_addons = '/?current=false'
request = requests.get('%s%s%s' % (api_url, end_point_plus_aux, url_addons), headers=headers)
response = json.loads(request.text)
print('Requested url: '+'%s%s%s. Be aware of the name requested' % (api_url, end_point_plus_aux, url_addons))

Requested url: http://api.socib.es/data-platforms/icoast00/files/latest/?current=false. Be aware of the name requested


In [14]:
files_metadata = {k:v for k, v in response.items() if k != 'entries'} #skipping the deployment data for later
for k, v in response.items():
    if k == 'entries':
        files_metadata[k] = len(v) 
HTML(json2html.convert(json=files_metadata))

type,Glider
variables,target_longitudetarget_latitudecurrent_target_longitudecurrent_target_latitudenorthward_sea_water_velocityeastward_sea_water_velocityinternal_pressurebooleandistancevoltagelocation_classiridium_phone_numbertransmission_retriesnumber_of_divetransmission_durationplatform_orientationplatform_speed_wrt_groundelectrical_comsumptionmission_duration
is_active,False
name,icoast00
entries,2


For every netCDF retourned you have a number of services to download, access and explore a given data-platform netCDFs (so-called entries):

In [15]:
files_metadata = {k:v for k, v in response.items()}
HTML(json2html.convert(json=files_metadata))